In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [2]:
ticker = 'SPY'
data = yf.download(ticker, start='2022-01-01', end='2025-03-15', progress=False)['Close']['SPY']

YF.download() has changed argument auto_adjust default to True


In [3]:
daily_returns = data.pct_change().dropna()
sigma = daily_returns.std() * np.sqrt(252)

In [11]:
S0 = data.iloc[-1]
r = 0.05 
# Días al vencimiento
T = 252
dt = 1 / T

num_simulations = 10000

Z = np.random.normal(0, 1, (num_simulations, T))

# Matriz de precios simulados del subyacente
# N filas x Pasos + 1 por S0
price_matrix = np.zeros((num_simulations, T+1))

In [12]:
barrier_price = S0

for i in range(num_simulations):
    price_path = [S0]
    barrier_breached = False

    for dia in range(1, T+1):
        St = price_path[-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z[i, dia-1])
        price_path.append(St)

        if St > barrier_price:
            barrier_breached = True
            break

        if not barrier_breached:
            price_matrix[i, :] = 0
        
        else:
            price_matrix[i, :] = price_path